<a href="https://colab.research.google.com/github/giorginolab/OpenMM-Tutorial-UniPD-2023/blob/main/OpenMM_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab-specific instructions start here

In [ ]:
# Here we use a Conda environment inside Google Colab. Blocks specific for Colab
# (like this one) mention "condacolab". On "normal" platforms the procedure
# for installation may be different - you need to check the system's documentation.  

# Colab notebooks are "brittle": in the course of time Colab is updated
# and dependencies no longer work properly. Proper HPC platforms are more
# stable (and supported)

# After executing this cell, Colab restarts.

!pip install -q condacolab   
import condacolab
condacolab.install_miniforge()

In [ ]:
# Verify Python version
import sys
sys.version

In [ ]:
import condacolab
condacolab.check()

In [ ]:
# Colab-specific workaround for a weird error upon shell escape:
#   NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Generic installation instructions

In [ ]:
# Verify GPU availability and type. If you get an error, check that 
# "Runtime / Runtime type / GPU" is selected.
!nvidia-smi

In [ ]:
# Install OpenMM. Takes a long time.
!conda install -q -c conda-forge openmm pdbfixer

# Tests

In [ ]:
# A quick test
import openmm.testInstallation
openmm.testInstallation.main()

In [ ]:
!(cd /usr/local/share/openmm/examples; python benchmark.py)

In [ ]:
# A more realistic benchmark. Note the "ns_per_day" figure
!(cd $CONDA_PREFIX/share/openmm/examples; python benchmark.py --platform OpenCL --test pme --seconds 5  --precision mixed)

In [ ]:
# CUDA (NVIDIA GPU) is the fastest platform. You can the others with...
# !(cd /usr/local/share/openmm/examples; python benchmark.py --platform CUDA --test pme --seconds 5  --precision mixed)
# !(cd /usr/local/share/openmm/examples; python benchmark.py --platform OpenCL --test pme --seconds 5  --precision mixed)
# !(cd /usr/local/share/openmm/examples; python benchmark.py --platform CPU --test pme --seconds 5  --precision mixed)

# Here begins the simulations tutorial proper 

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from pdbfixer import *
from sys import stdout

## Download, fix missing atoms, solvate

Can also be done on the command line with the `pdbfixer` executable.

In [ ]:
# Retrieve the structure from the RCSB 
fixer = PDBFixer(pdbid="6H1F")

# Add missing (unresolved) residues. We don't want to model anything.
fixer.findMissingResidues()
fixer.missingResidues = {}
# fixer.addMissingResidues()

# Add missing (unresolved) atoms
fixer.findMissingAtoms()
fixer.addMissingAtoms()

# Protonate (roughly) at chosen pH
fixer.addMissingHydrogens(pH=7.0)

# Explicit solvent: 10 nm^3 box
fixer.addSolvent(boxSize=10 * Vec3(1, 1, 1))

# Save the file so it can be inspected
PDBFile.writeFile(fixer.topology, fixer.positions, open("6H1F-fixed.pdb", "w"))

## Modeling

Modelling step. Sometimes unnecessary. Here it is 
needed to remove an "SCN" (THIOCYANATE ION) residue.


In [ ]:
# There is an "SCN" residue to remove
modeller = Modeller(fixer.topology, fixer.positions)

res_SCN = [r for r in modeller.topology.residues() if r.name == "SCN"]
modeller.delete(res_SCN)

PDBFile.writeFile(
    modeller.topology, modeller.positions, open("6H1F-modelled.pdb", "w"), keepIds=True
)


## Create integration-related objects

In [ ]:
# The FF object holds the parameters
forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

In [ ]:
# This specifies the system to be simulated. 
system = forcefield.createSystem(
    modeller.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1 * nanometer,
    constraints=HBonds,
)

In [ ]:
# Specify the integrator: temperature, relaxation time, timestep (important)
integrator = LangevinMiddleIntegrator(300 * kelvin, 1 / picosecond, 0.004 * picoseconds)

In [ ]:
# The barostat is added to the system so that density is controlled 
# in addition to temperature.

# Pressure, Temperature (only used for calculation),
# Frequency (how frequently the system should update the box size)
barostat = MonteCarloBarostat(1.0 * atmosphere, 300.0 * kelvin, 25)

system.addForce(barostat)


In [ ]:
# Combines the molecular topology, system, and integrator
# to begin a new simulation.
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

## Minimize energy

In [ ]:
# Perform local energy minimization
print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=500)


# Write the minimized coordinates (for checking)
PDBFile.writeFile(
    simulation.topology,
    simulation.context.getState(getPositions=True).getPositions(),
    open("6H1F-minimized.pdb", "w"),
    keepIds=True,
)

## Integrate

In [ ]:
Nsteps = 5000

In [ ]:
# When the simulation runs, it will write the trajectory to a file called "output.pdb"
simulation.reporters.append(
    DCDReporter("output.dcd", reportInterval=1000, enforcePeriodicBox=True)
)


In [ ]:
# Also report infomation to the screen as the simulation runs
simulation.reporters.append(
    StateDataReporter(
        stdout,
        100,
        step=True,
        time=True,
        potentialEnergy=True,
        kineticEnergy=True,
        totalEnergy=True,
        temperature=True,
        volume=True,
        density=True,
        progress=True,
        remainingTime=True,
        speed=True,
        elapsedTime=True,
        separator=" ",
        totalSteps=Nsteps,
    )
)



In [ ]:
# Finally run the simulation for the given timesteps
print("Running simulation...")
simulation.step(Nsteps)

# Results

The simulation is completed. Now download the minimized PDB file (that gives the starting coordinates and the identity of the atoms) and the DCD file (a binary file, providing the trajectory, i.e. a series of snapshots of the coordinates). They can be best visualized locally on e.g. PyMol or VMD.


# Visualize

Here we use a cloud-based viewer, but you should better download the results and use VMD.

In [ ]:
!conda install -q MDAnalysis mdtraj
!pip install py3Dmol 
import py3Dmol

In [ ]:
!mdconvert  -o output.pdb -t 6H1F-minimized.pdb output.dcd

In [ ]:
view = py3Dmol.view(width=400, height=300)
view.addModel(open('6H1F-minimized.pdb', 'r').read(),'pdb')
view.addStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
